## HNSW (Hierarchical Navigable Small World)
This notebook demonstrates how to use the `hnswlib` library to perform efficient **k-nearest neighbors (k-NN)** searches on large datasets of embeddings, specifically for images and academic papers.

### 1. Initialization and Setup
This section installs the necessary Python libraries, clones a repository containing embedding utilities *(EmbedX)*, and installs its dependencies.

In [1]:
!pip install gdown hnswlib
!git clone https://github.com/huynguyen6906/EmbedX.git
!pip install -r EmbedX/requirements.txt
!cd EmbedX && pip install .

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp312-cp312-linux_x86_64.whl size=227427 sha256=4420f2c182857cce9f6d440191bae5e398ae360f7f075e71d8f9f34f311e3145
  Stored in directory: /home/huynguyen/.cache/pip/wheels/ac/39/b3/cbd7f9cbb76501d2d5fbc84956e70d0b94e788aac87bda465e
Successfully built hnswlib
Cloning into 'EmbedX'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 71 (delta 40), reused 49 (delta 18), pack-reused 0 (from 0)
Receiving objects: 100% (71/71), 9.51 KiB | 1.90 MiB/s, done.
Resolving deltas: 100% (40/40), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 29.0 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 34.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 30.4 M

In [1]:
import hnswlib
import gdown
import h5py
import clip
import embedx
import json
import torch
from sentence_transformers import SentenceTransformer
import os
import numpy as np

### 2. Download Cache
This section checks for the existence of cached files containing pre-computed embeddings and HNSW indices. If any are missing, it uses `gdown` to download them from Google Drive using their IDs.

In [2]:
# Ensure the local directory for caching files exists.
os.makedirs(".cache", exist_ok=True)

if not os.path.isfile(".cache/Papers_Embedbed_0-1000000.h5"):
    # If not present, download the snapshot file using its Google Drive ID.
    gdown.download(id='1hMNur1U24ULce6osanfrePuCAz5uIvc3', output='.cache/Papers_Embedbed_0-1000000.h5', quiet=False)

if not os.path.isfile(".cache/hnsw_paper_index.bin"):
    # If not present, download the snapshot file using its Google Drive ID.
    gdown.download(id='1RR1o_eX9LKv_3pd9Iz1y4xB90pQIpUuX', output='.cache/hnsw_paper_index.bin', quiet=False)

if not os.path.isfile(".cache/hnsw_paper_index.json"):
    # If not present, download the snapshot file using its Google Drive ID.
    gdown.download(id='1H_BNLy9fXYwaOCXvZBRawN8S0w_KuGIK', output='.cache/hnsw_paper_index.json', quiet=False)

if not os.path.isfile(".cache/Image_Embedded.h5"):
    # If not present, download the snapshot file using its Google Drive ID.
    gdown.download(id='12R_DHBMOVFVEaPrjBEQXOd7tb14Isb1D', output='.cache/Image_Embedded.h5', quiet=False)

if not os.path.isfile(".cache/hnsw_image_index.bin"):
    # If not present, download the snapshot file using its Google Drive ID.
    gdown.download(id='1vWbJFLTdFnT6N2cnpMDA7c85JCfrY-QH', output='.cache/hnsw_image_index.bin', quiet=False)

if not os.path.isfile(".cache/hnsw_image_index.json"):
    # If not present, download the snapshot file using its Google Drive ID.
    gdown.download(id='16F8VRuKZb4SJ0KLa1Oh281RvdmQzj6Ft', output='.cache/hnsw_image_index.json', quiet=False)

### 3. Search Image by HNSW
This section loads the image embedding data and the HNSW index configuration to perform image search based on text queries.

Reading image URLs and their corresponding embedding vectors from the `.h5` file.

In [3]:
image = h5py.File(".cache/Image_Embedded.h5", "r")

Reading the essential configuration parameters (`dim`, `max_element`, `ef`) for the HNSW index from the JSON file.

In [4]:
image_index_config = ".cache/hnsw_image_index.json"
dim_image = 0
max_element_image = 0
ef_image = 0
with open(image_index_config, 'r') as f:
    data = json.load(f)
    dim_image = data[0]
    max_element_image = data[1]
    ef_image = data[2]

Initializing and loading the pre-built HNSW index from the binary (`.bin`) file.

In [5]:
image_index_file = ".cache/hnsw_image_index.bin"

if os.path.exists(image_index_file):
    image_search = hnswlib.Index(space='cosine', dim=dim_image)
    image_search.load_index(image_index_file, max_elements=max_element_image) 
    image_search.set_ef(ef_image)   
else:
    print("❌ ERROR: file is not exist.")

Performing text queries, embedding them into vectors using `embedx`, and searching for the top 10 nearest images (k=10) in the HNSW index.

In [6]:
Text_queries = ["dog", "cat", "math"]
queries = [embedx.image.embed_Text(Text_queries[i]) for i in range(len(Text_queries))]
indices, distances = image_search.knn_query(queries, k=10)
for i in range(len(Text_queries)):
  print(Text_queries[i])
  for idx in indices[i]:
    print(image["urls"][idx])
del indices
del distances

dog
b'https://c4.staticflickr.com/1/87/243731034_4d1701e1c0_o.jpg'
b'https://farm3.staticflickr.com/6044/6323964074_9a8ddb39db_o.jpg'
b'https://farm7.staticflickr.com/3459/3979846273_c99f81d4b1_o.jpg'
b'https://farm6.staticflickr.com/3868/14679717335_96e127605a_o.jpg'
b'https://c5.staticflickr.com/3/2719/4079091347_a639a4b302_o.jpg'
b'https://c8.staticflickr.com/8/7605/16712433347_336724d690_o.jpg'
b'https://farm6.staticflickr.com/3105/4552760734_936cf97707_o.jpg'
b'https://farm5.staticflickr.com/38/81350468_c7749aef0e_o.jpg'
b'https://farm3.staticflickr.com/2867/10421179043_da78ca3485_o.jpg'
b'https://farm5.staticflickr.com/155/395663099_8fcd626424_o.jpg'
cat
b'https://c8.staticflickr.com/6/5343/9308613838_39b958e322_o.jpg'
b'https://c5.staticflickr.com/8/7445/16486598725_557e78555a_o.jpg'
b'https://farm3.staticflickr.com/6014/5965911714_38e50ec21d_o.jpg'
b'https://c4.staticflickr.com/3/2577/3871309668_90a410a28b_o.jpg'
b'https://c5.staticflickr.com/1/178/389242128_df4f33ece3_o.jpg'
b

Deleting variables to free up memory after the image search section.

In [11]:
del image_index_config
del image_search
del image_index_file
del dim_image
del max_element_image
del ef_image
image.close()

### 4. Search Paper by HNSW (SEARCH PAPER BY HNSW)
This section loads the academic paper embedding data and the HNSW index configuration to search for relevant papers based on text queries.

Reading paper URLs and their corresponding embedding vectors from the `.h5` file.

In [7]:
paper = h5py.File(".cache/Papers_Embedbed_0-1000000.h5", "r")

Reading the essential configuration parameters (`dim`, `max_element`, `ef`) for the HNSW index from the JSON file.

In [8]:
paper_index_config = ".cache/hnsw_paper_index.json"
dim_paper = 0
max_element_paper = 0
ef_paper = 0
with open(paper_index_config, 'r') as f:
    data = json.load(f)
    dim_paper = data[0]
    max_element_paper = data[1]
    ef_paper = data[2]

Initializing and loading the pre-built HNSW index from the binary (`.bin`) file.

In [9]:
paper_index_file = ".cache/hnsw_paper_index.bin"

if os.path.exists(paper_index_file):
    paper_search = hnswlib.Index(space='cosine', dim=dim_paper)
    paper_search.load_index(paper_index_file, max_elements=max_element_paper) 
    paper_search.set_ef(ef_paper)   
else:
    print("❌ ERROR: file is not exist.")

Performing text queries, embedding them into vectors using `embedx`, and searching for the top 10 nearest papers (k=10) in the HNSW index.

In [10]:
Text_queries = ["computer", "chemical"]
queries = [embedx.text.embed_Text(Text_queries[i]) for i in range(len(Text_queries))]
indices, distances = paper_search.knn_query(queries, k=10)
for i in range(len(Text_queries)):
  print(Text_queries[i])
  for idx in indices[i]:
    print(paper["urls"][idx])
del indices
del distances

computer
b'https://arxiv.org/pdf/1309.5737.pdf'
b'https://arxiv.org/pdf/1107.3893.pdf'
b'https://arxiv.org/pdf/1107.4217.pdf'
b'https://arxiv.org/pdf/0903.4286.pdf'
b'https://arxiv.org/pdf/1304.1428.pdf'
b'https://arxiv.org/pdf/1712.09404.pd'
b'https://arxiv.org/pdf/1312.2447.pdf'
b'https://arxiv.org/pdf/1703.02944.pd'
b'https://arxiv.org/pdf/1202.5944.pdf'
b'https://arxiv.org/pdf/0910.3440.pdf'
chemical
b'https://arxiv.org/pdf/1606.04381.pd'
b'https://arxiv.org/pdf/1207.3242.pdf'
b'https://arxiv.org/pdf/1307.6360.pdf'
b'https://arxiv.org/pdf/1007.0818.pdf'
b'https://arxiv.org/pdf/0708.1826.pdf'
b'https://arxiv.org/pdf/1301.0833.pdf'
b'https://arxiv.org/pdf/0906.0871.pdf'
b'https://arxiv.org/pdf/1211.3163.pdf'
b'https://arxiv.org/pdf/1601.05356.pd'
b'https://arxiv.org/pdf/0901.0318.pdf'


Deleting variables to free up memory after the paper search section.

In [12]:
del paper_index_config
del paper_search
del paper_index_file
del dim_paper
del max_element_paper
del ef_paper
paper.close()